<a href="https://colab.research.google.com/github/Praneeth-gamidi/Book-Recommendation-System/blob/main/Book_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from math import sqrt
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="openpyxl.utils")
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl.styles")
books = pd.read_excel('/content/books.xlsx')
ratings = pd.read_excel('/content/ratings.xlsx')
users = pd.read_excel('/content/users.xlsx')

book_data = []
for _, rating_row in ratings.iterrows():
    for _, book_row in books.iterrows():
        if rating_row['book_id'] == book_row['book_id']:
            book_data.append({**rating_row, **book_row})

book_data = pd.DataFrame(book_data)
full_data = []
for _, book_row in book_data.iterrows():
    for _, user_row in users.iterrows():
        if book_row['user_id'] == user_row['user_id']:
            full_data.append({**book_row, **user_row})

full_data = pd.DataFrame(full_data)

unique_categories = full_data['category'].unique()
category_encoded = pd.DataFrame([[1 if full_data['category'][i] == category else 0 for category in unique_categories] for i in range(len(full_data))], columns=[f"category_{category}" for category in unique_categories])

feature_matrix = pd.concat([full_data[['user_id', 'book_id', 'rating']].reset_index(drop=True), category_encoded.reset_index(drop=True)], axis=1)

user_book_matrix = {}
for _, row in feature_matrix.iterrows():
    user_id, book_id, rating = row['user_id'], row['book_id'], row['rating']
    if user_id not in user_book_matrix:
        user_book_matrix[user_id] = {}
    user_book_matrix[user_id][book_id] = rating

book_ids = feature_matrix['book_id'].unique()
for user_ratings in user_book_matrix.values():
    for book_id in book_ids:
        if book_id not in user_ratings:
            user_ratings[book_id] = 0

def euclidean_distance(vec1, vec2):
    return sqrt(sum((vec1[i] - vec2[i]) ** 2 for i in range(len(vec1))))

def recommend_books(user_id, book_title, n_recommendations=5):
    if book_title not in books['title'].values:
        return "Book not found in the dataset."

    book_id = books[books['title'] == book_title]['book_id'].values[0]
    if book_id not in book_ids:
        return "Book not found in the ratings data."

    user_age = users[users['user_id'] == user_id]['age'].values[0]
    book_ratings = [user_book_matrix[u].get(book_id, 0) for u in user_book_matrix]

    distances = []
    for other_book_id in book_ids:
        if other_book_id == book_id:
            continue
        other_book_ratings = [user_book_matrix[u].get(other_book_id, 0) for u in user_book_matrix]
        distance = euclidean_distance(book_ratings, other_book_ratings)
        distances.append((distance, other_book_id))
    distances.sort()
    recommendations = []
    for _, similar_book_id in distances[:n_recommendations + 1]:
        similar_book_title = books[books['book_id'] == similar_book_id]['title'].values[0]
        book_age_group = books[books['book_id'] == similar_book_id]['target_age_group'].values[0]
        if user_age >= book_age_group:
            recommendations.append(similar_book_title)
            if len(recommendations) >= n_recommendations:
                break

    return recommendations
user_id = int(input("Enter Your user ID: "))
book_title = input("Enter book title for which you need similar books: ")
n_recommendations = int(input("Enter number of recommendations:"));
recommended_books = recommend_books(user_id, book_title, n_recommendations)
print(f"Books similar to '{book_title}' for user with ID {user_id}:\n", recommended_books)

Enter Your user ID: 3
Enter book title for which you need similar books: something
Enter number of recommendations:2
Books similar to 'something' for user with ID 3:
 Book not found in the dataset.
